In [1]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import optuna

from torch.utils.data import DataLoader
from src.data import MovingMnistDataset
import torch.optim as optim
import time

from src.dl_models.unet import UNet
from src import model, visualization, evaluate, data, preprocessing,train


In [26]:
class Objective_optuna(object):
    def __init__(self, path, csv, device):
        # Called when the study is created
        self.device = device
        self.path = path
        self.csv = csv

    def __call__(self, trial):
        # Called on each trial

        # Hyperparameters
        n_channels=3
        n_classes=1

        # Load data
        mnist_dataset = MovingMnistDataset(path=self.path, csv=self.csv, shuffle=True)
        loader = DataLoader(mnist_dataset, batch_size=10)

        # Initialize network
        model = UNet(n_channels=n_channels, n_classes=n_classes, bilinear=True).to(device)
        model.train()

        # Hyperparameters
        lr = trial.suggest_float("lr", 1e-5, 1e-1)
        optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
        optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
        criterion_name =  trial.suggest_categorical("criterion", ["MSELoss", "L1Loss"])
        criterion = getattr(nn, criterion_name)()

        loss_history = train.train_model(
            model,
            loader,
            criterion,
            optimizer,
            self.device,
            curr_epoch=0,
            loss_history=[],
            train_for=2,
            verbose=True,
            checkpoint_every=0,
            print_cuda_mem=False
        )
        return loss_history[-1]

path = 'C:/Users/Ignacio/Desktop/moving_mnist/moving_mnist_train/'
csv = 'C:/Users/Ignacio/Desktop/moving_mnist/small_train.csv'

# Set device
USE_CUDA = True
device = torch.device('cuda' if torch.cuda.is_available()
                    and USE_CUDA else 'cpu')
print(f'Using {device}')

# Execute an optimization by using an `Objective` instance.
study = optuna.create_study()
study.optimize(Objective_optuna(path, csv, device), n_trials=3)

print("\nBest trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-05-26 00:40:04,106] A new study created in memory with name: no-name-b58a195d-cdb3-4524-b669-1ad6953acccd
Using cuda
EPOCH: 1 Time elapsed: 19.73 secs.
EPOCH: 2 [I 2021-05-26 00:40:43,451] Trial 0 finished with value: 0.010364263318479061 and parameters: {'lr': 0.014479582599625403, 'optimizer': 'RMSprop', 'criterion': 'MSELoss'}. Best is trial 0 with value: 0.010364263318479061.
Time elapsed: 19.44 secs.
EPOCH: 1 Time elapsed: 20.49 secs.
EPOCH: 2 [I 2021-05-26 00:41:24,494] Trial 1 finished with value: 0.014701028354465961 and parameters: {'lr': 0.06626875632783874, 'optimizer': 'Adam', 'criterion': 'MSELoss'}. Best is trial 0 with value: 0.010364263318479061.
Time elapsed: 20.38 secs.
EPOCH: 1 Time elapsed: 17.60 secs.
EPOCH: 2 [I 2021-05-26 00:41:59,929] Trial 2 finished with value: 0.03419813513755798 and parameters: {'lr': 0.04689562237250156, 'optimizer': 'SGD', 'criterion': 'L1Loss'}. Best is trial 0 with value: 0.010364263318479061.
Time elapsed: 17.66 secs.

Best tria